In [35]:
import pandas as pd
import numpy as np

df = pd.read_csv('orders.csv')
temp_df = df

In [40]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler

class Preprocessor : 
    def __init__ (self, df):
        self.df = df.copy()
        self.k=5
        self.original_df = df.copy()
    def handle_missing_values (self) :
        self.df.fillna(0, inplace=True)
      
    def transform (self) : 
        self.handle_missing_values()
        self.time_series()
        self.rare_label_encoding()
        self.min_max_scaler()
        self.submission_df_full = pd.merge(self.original_df['ID_Customer'] , self.df, on='ID_Customer', how='left')
        self.drop_columns()
        return self.submission_df_full
    
    def drop_columns(self) :
        self.submission_df_full.drop(columns=['ID_Customer'],inplace=True)
        
    def rare_label_encoding(self):
        top_k_labels = self.df['city_name_fa'].value_counts().nlargest(self.k).index.tolist()
        
        self.city_map = {label: i + 1 for i, label in enumerate(top_k_labels)}
        
        self.df['city_encoded'] = self.df['city_name_fa'].apply(lambda x: self.city_map.get(x, self.k + 1))
        
        self.df.drop(columns=['city_name_fa'], inplace=True)
        
    def min_max_scaler(self):
        scaler = StandardScaler()
        self.df[['Amount_Gross_Order','Recency']] = scaler.fit_transform(self.df[['Amount_Gross_Order','Recency']])        
    
    def time_series(self):
        self.df['DateTime_CartFinalize'] = pd.to_datetime(self.df['DateTime_CartFinalize'])

        latest_date = self.df['DateTime_CartFinalize'].max()
        last_order = self.df.groupby('ID_Customer')['DateTime_CartFinalize'].max()
        recency = (latest_date - last_order).dt.days
        recency = recency.reset_index().rename(columns={'DateTime_CartFinalize': 'Recency'})

        customer_df = self.df.groupby('ID_Customer').agg(
            last_order=('DateTime_CartFinalize', 'max'),
            first_order=('DateTime_CartFinalize', 'min'),
            order_count=('DateTime_CartFinalize', 'count')
        )

        customer_df['last_order'] = pd.to_datetime(customer_df['last_order'])
        customer_df['first_order'] = pd.to_datetime(customer_df['first_order'])

        total_days = (customer_df['last_order'] - customer_df['first_order']).dt.days
        customer_df['frequency_per_month'] = customer_df['order_count'] / (total_days +5 + 1e-6)
        customer_df=customer_df.reset_index()
        customer_df.drop(columns = ['first_order','last_order','order_count'],inplace=True)
        
        # a dataframe with CUSTOMER_ID and RECENCY and FREQUENCY
        together_df = customer_df.merge(recency, on='ID_Customer' , how = 'inner')
        # a dataframe with ID_CUSTOMER and QUANTITY_ITEM
        df2 = self.df.groupby('ID_Customer')['Quantity_item'].sum().reset_index()
        # a dataframe with ID_CUSTOMER and AMOUNT_GROSS_Order
        df3 = self.df.groupby('ID_Customer')['Amount_Gross_Order'].sum().reset_index()
        # a dataframe with ID_CUSTOMER and city_name_fa
        df_unique = self.df.drop_duplicates(subset='ID_Customer', keep='first')
        df_unique.drop(columns = ['ID_Order','ID_Item','DateTime_CartFinalize','Amount_Gross_Order','Quantity_item'] , inplace=True)
        alltogether_df = together_df.merge(df2,on='ID_Customer',how='inner')
        alltogether_df = alltogether_df.merge(df3,on='ID_Customer',how='inner')
        alltogether_df=alltogether_df.merge(df_unique , on='ID_Customer',how='inner')
        self.df = alltogether_df


In [43]:
%%writefile preprocessor.py

import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler

class Preprocessor : 
    def __init__ (self, df):
        self.df = df.copy()
        self.k=5
        self.original_df = df.copy()
    def handle_missing_values (self) :
        self.df.fillna(0, inplace=True)
      
    def transform (self) : 
        self.handle_missing_values()
        self.time_series()
        self.rare_label_encoding()
        self.min_max_scaler()
        self.submission_df_full = pd.merge(self.original_df['ID_Customer'] , self.df, on='ID_Customer', how='left')
        self.drop_columns()
        return self.submission_df_full
    
    def drop_columns(self) :
        self.submission_df_full.drop(columns=['ID_Customer'],inplace=True)
        
    def rare_label_encoding(self):
        top_k_labels = self.df['city_name_fa'].value_counts().nlargest(self.k).index.tolist()
        
        self.city_map = {label: i + 1 for i, label in enumerate(top_k_labels)}
        
        self.df['city_encoded'] = self.df['city_name_fa'].apply(lambda x: self.city_map.get(x, self.k + 1))
        
        self.df.drop(columns=['city_name_fa'], inplace=True)
        
    def min_max_scaler(self):
        scaler = StandardScaler()
        self.df[['Amount_Gross_Order','Recency']] = scaler.fit_transform(self.df[['Amount_Gross_Order','Recency']])        
    
    def time_series(self):
        self.df['DateTime_CartFinalize'] = pd.to_datetime(self.df['DateTime_CartFinalize'])

        latest_date = self.df['DateTime_CartFinalize'].max()
        last_order = self.df.groupby('ID_Customer')['DateTime_CartFinalize'].max()
        recency = (latest_date - last_order).dt.days
        recency = recency.reset_index().rename(columns={'DateTime_CartFinalize': 'Recency'})

        customer_df = self.df.groupby('ID_Customer').agg(
            last_order=('DateTime_CartFinalize', 'max'),
            first_order=('DateTime_CartFinalize', 'min'),
            order_count=('DateTime_CartFinalize', 'count')
        )

        customer_df['last_order'] = pd.to_datetime(customer_df['last_order'])
        customer_df['first_order'] = pd.to_datetime(customer_df['first_order'])

        total_days = (customer_df['last_order'] - customer_df['first_order']).dt.days
        customer_df['frequency_per_month'] = customer_df['order_count'] / (total_days +5 + 1e-6)
        customer_df=customer_df.reset_index()
        customer_df.drop(columns = ['first_order','last_order','order_count'],inplace=True)
        
        # a dataframe with CUSTOMER_ID and RECENCY and FREQUENCY
        together_df = customer_df.merge(recency, on='ID_Customer' , how = 'inner')
        # a dataframe with ID_CUSTOMER and QUANTITY_ITEM
        df2 = self.df.groupby('ID_Customer')['Quantity_item'].sum().reset_index()
        # a dataframe with ID_CUSTOMER and AMOUNT_GROSS_Order
        df3 = self.df.groupby('ID_Customer')['Amount_Gross_Order'].sum().reset_index()
        # a dataframe with ID_CUSTOMER and city_name_fa
        df_unique = self.df.drop_duplicates(subset='ID_Customer', keep='first')
        df_unique.drop(columns = ['ID_Order','ID_Item','DateTime_CartFinalize','Amount_Gross_Order','Quantity_item'] , inplace=True)
        alltogether_df = together_df.merge(df2,on='ID_Customer',how='inner')
        alltogether_df = alltogether_df.merge(df3,on='ID_Customer',how='inner')
        alltogether_df=alltogether_df.merge(df_unique , on='ID_Customer',how='inner')
        self.df = alltogether_df


Overwriting preprocessor.py


In [93]:
preprocessor = Preprocessor(df)
X = preprocessor.transform()
customer_df =X

C:\Users\ahmad\AppData\Local\Temp\ipykernel_8548\1939181444.py:68: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_unique.drop(columns = ['ID_Order','ID_Item','DateTime_CartFinalize','Amount_Gross_Order','Quantity_item'] , inplace=True)


In [108]:

N_CLUSTERS = 3

from sklearn.cluster import KMeans
model = KMeans(n_clusters=N_CLUSTERS , random_state=42 , n_init = 10)
customer_df['Cluster']=model.fit_predict(X)


In [109]:
submission = pd.DataFrame(customer_df['Cluster'])
submission.to_csv('submission.csv', index=False)
submission

,Cluster
0,0
1,0
2,0
3,0
4,0
...,...
199995,0
199996,0
199997,0
199998,0


<h3 align=right style="line-height:200%;font-family:vazir;color:#0099cc">
<font face="vazirmatn" color="#0099cc">
معیار ارزیابی
</font>
</h3>

<p dir=rtl style="direction: rtl; text-align: justify; line-height:200%; font-family:vazir; font-size:medium">
<font face="vazirmatn" size=3>
    معیاری که برای ارزیابی عملکرد مدل انتخاب کرده‌ایم، <code>silhouette score</code> نام دارد.
    <br>
</font>
</p>

In [110]:
from sklearn.metrics import silhouette_score

# Calculate silhouette score using existing data and cluster labels
sil_score = silhouette_score(Preprocessor(pd.read_csv('orders.csv')).transform(),pd.read_csv('submission.csv').Cluster)
print(f'Silhouette Score: {sil_score:.3f}')


C:\Users\ahmad\AppData\Local\Temp\ipykernel_8548\1939181444.py:68: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_unique.drop(columns = ['ID_Order','ID_Item','DateTime_CartFinalize','Amount_Gross_Order','Quantity_item'] , inplace=True)


Silhouette Score: 0.849


In [111]:
print( 100 * (sil_score + 1 ) / 2 )

92.46149668904134


In [113]:
import zipfile
import joblib

def compress(file_names):
    print("File Paths:")
    print(file_names)
    compression = zipfile.ZIP_DEFLATED
    with zipfile.ZipFile("result.zip", mode="w") as zf:
        for file_name in file_names:
            zf.write('./' + file_name, file_name, compress_type=compression)

joblib.dump(model, 'model')
submission.to_csv('submission.csv', index=False)
file_names = ['Clustering.ipynb', 'submission.csv', 'model', 'preprocessor.py']
compress(file_names)

File Paths:
['Clustering.ipynb', 'submission.csv', 'model', 'preprocessor.py']
